In [1]:
import os
import pandas as pd
from itertools import product
from datetime import datetime

In [10]:
def func1(year: str, month: str):
    # 설정
    data_dir = './data/'
    file_name = f'{year}년_{month}월_자동차_등록자료_통계.xlsx'
    path = os.path.join(data_dir, file_name)

    # 1. 데이터 로드 및 불필요한 행/열 제거
    df = pd.read_excel(path, skiprows=4, sheet_name=0)
    # 두 번째 열(index 1) 제외 및 시도별 행 추출
    df = df.drop(df.columns[1], axis=1)

    # 2. 컬럼명 재설정 (MultiIndex 활용 준비)
    car_type = ['승용', '승합', '화물', '특수', '총계']
    car_use = ['관용', '자가용', '영업용', '계']
    new_cols = ['시도별'] + [f"{t}_{u}" for t, u in product(car_type, car_use)]
    df.columns = new_cols

    # 3. '계', '총계', '합계'가 포함된 행/열 미리 제거
    ignores = ['총계', '계','월간증감', '년간증감']

    # '시도별' 열에서 합계 행 제외
    df = df[~df['시도별'].isin(ignores)].dropna()

    # 4. Unpivot (Wide to Long) - 이중 루프 대체
    res_df = df.melt(id_vars=['시도별'], var_name='type_use', value_name='count')

    # 5. 컬럼 분리 및 날짜 추가
    res_df[['차종', '용도']] = res_df['type_use'].str.split('_', expand=True)
    res_df['연월'] = f"{year}-{month}"

    # 6. 불필요한 항목('계') 필터링 및 컬럼 순서 정리
    res_df = res_df[~res_df['차종'].isin(ignores) & ~res_df['용도'].isin(ignores)]
    res_df = res_df[['연월', '시도별', '차종', '용도', 'count']]

    return res_df

In [13]:
YEARS = [str(y) for y in range(2016, 2027)]
MONTHS = [f'0{str(m)}' if m < 10 else str(m) for m in range(1, 13)]
lst_df = []

for year in YEARS:
    print('\nProcessing data from', year)
    for month in MONTHS:
        print(month, end=' ')
        try:
            lst_df.append(func1(year, month))
        except:
            break
        # lst_df.append(func1(year, month))

res_df = pd.concat(lst_df)
res_df.to_csv('stat_region.csv', index=False)


Processing data from 2016
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2017
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2018
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2019
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2020
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2021
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2022
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2023
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2024
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2025
01 02 03 04 05 06 07 08 09 10 11 12 
Processing data from 2026
01 02 